In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]

'The Blackwell GPU features fifth-generation Tensor Cores, fourth-generation RT Cores, and supports DLSS 4 and NVLink-5. It has a high power limit and advanced memory technologies.'

In [3]:
# Regular Search 
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

In [4]:
# Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

C:\Users\Chaudhary Computers\AppData\Local\Temp\ipykernel_13840\2099449103.py:8: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


https://zhidao.baidu.com/question/10035040.html
https://www.current-news.co.uk/energy-uk-urges-government-to-unlock-growth-through-energy-industry-regulation/
https://www.current-news.co.uk/current±-webinars-of-2023/
https://zhidao.baidu.com/question/2022512578010206988.html
https://www.current-news.co.uk/current±-disrupters-deep-greens-mark-bjornsgaard-on-using-data-servers-to-heat-swimming-pools/
https://forum.wordreference.com/threads/possession-of-a-current-licence.3777513/


In [5]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [6]:
# Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://zhidao.baidu.com/question/10035040.html


<body>
<div class="timeout hide-callback">
<div class="timeout-img"></div>
<div class="timeout-title">ç½ç»ä¸ç»åï¼è¯·ç¨åéè¯</div>
<button class="timeout-button" type="button">è¿åé¦é¡µ</button>
</div>
<div class="timeout-feedback hide-callback">
<div class="timeout-feedback-icon"></div>
<p class="timeout-feedback-title">é®é¢åé¦</p>
</div>
<script src="https://ppui-static-wap.cdn.bcebos.com/static/touch/js/mkdjump_v2_21d1ae1.js"></script>
</body>


In [7]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://zhidao.baidu.com/question/10035040.html


é®é¢åé¦


In [8]:
#Agentic Search
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1761744031, 'localtime': '2025-10-29 06:20'}, 'current': {'last_updated_epoch': 1761743700, 'last_updated': '2025-10-29 06:15', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 34, 'wind_dir': 'NE', 'pressure_mb': 1020.0, 'pressure_in': 30.13, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 25, 'feelslike_c': 10.9, 'feelslike_f': 51.6, 'windchill_c': 12.1, 'windchill_f': 53.8, 'heatindex_c': 13.0, 'heatindex_f': 55.4, 'dewpoint_c': 8.1, 'dewpoint_f': 46.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 11.3, 'gust_kph': 18.1}}


In [9]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)
# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1761744031, 'localtime': '2025-10-29 06:20'}, 'current': {'last_updated_epoch': 1761743700, 'last_updated': '2025-10-29 06:15', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 34, 'wind_dir': 'NE', 'pressure_mb': 1020.0, 'pressure_in': 30.13, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 25, 'feelslike_c': 10.9, 'feelslike_f': 51.6, 'windchill_c': 12.1, 'windchill_f': 53.8, 'heatindex_c': 13.0, 'heatindex_f': 55.4, 'dewpoint_c': 8.1, 'dewpoint_f': 46.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 11.3, 'gust_kph': 18.1}}
{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "Un